In [1]:
import pandas as pd
books = pd.read_csv("books_with_categories.csv")

In [2]:
import os, time, requests
from typing import Union, List
from dotenv import load_dotenv

load_dotenv()
HF_TOKEN = os.getenv("HUGGING_FACE_TOKEN") or os.getenv("HUGGINGFACEHUB_API_TOKEN")

class TextClassificationAPI:
    """Hugging Face Inference API wrapper emulating transformers.pipeline('text-classification')."""
    def __init__(self, model: str = "j-hartmann/emotion-english-distilroberta-base",
                 token: str | None = None,
                 timeout: int = 60):
        self.url = f"https://api-inference.huggingface.co/models/{model}"
        self.headers = {"Authorization": f"Bearer {token}"} if token else {}
        self.timeout = timeout

    def __call__(self, sequences: Union[str, List[str]], top_k: int | None = None):
        payload = {
            "inputs": sequences,
            "parameters": ({"top_k": top_k} if top_k is not None else {}),
            "options": {"wait_for_model": True},  # avoid cold-start 503s
        }
        # simple retry for 429/503
        for attempt in range(3):
            r = requests.post(self.url, headers=self.headers, json=payload, timeout=self.timeout)
            if r.status_code in (429, 503):
                time.sleep(2 * (attempt + 1))
                continue
            r.raise_for_status()
            data = r.json()
            # Normalize shape to mimic transformers:
            # - single string -> list[{"label","score"}, ...]
            # - list[str] -> list[list[{"label","score"}, ...]]
            if isinstance(sequences, str) and isinstance(data, list) and data and isinstance(data[0], list):
                return data[0]
            return data
        raise RuntimeError("Inference API unavailable after retries.")


In [3]:
classifier = TextClassificationAPI(token=HF_TOKEN)

In [4]:
classifier("I love this!", top_k=None)

[{'label': 'joy', 'score': 0.9771687984466553},
 {'label': 'surprise', 'score': 0.008528676815330982},
 {'label': 'neutral', 'score': 0.005764589179307222},
 {'label': 'anger', 'score': 0.004419777542352676},
 {'label': 'sadness', 'score': 0.002092391485348344}]

In [5]:
classifier("This is awful.", top_k=3)

[{'label': 'disgust', 'score': 0.9522968530654907},
 {'label': 'fear', 'score': 0.023751985281705856},
 {'label': 'anger', 'score': 0.011204187758266926}]

In [6]:
classifier(["Great book!", "Terrible ending."], top_k=None)

[[{'label': 'joy', 'score': 0.9561761617660522},
  {'label': 'fear', 'score': 0.9000383019447327}]]

In [7]:
books["description"][0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

In [8]:
classifier(books["description"][0])

[{'label': 'fear', 'score': 0.6548415422439575},
 {'label': 'neutral', 'score': 0.1698518991470337},
 {'label': 'sadness', 'score': 0.1164088100194931},
 {'label': 'surprise', 'score': 0.02070065215229988},
 {'label': 'disgust', 'score': 0.019100727513432503}]

In [9]:
classifier(books["description"][0].split("."))

[[{'label': 'surprise', 'score': 0.7296026349067688},
  {'label': 'neutral', 'score': 0.44937047362327576},
  {'label': 'neutral', 'score': 0.6462168097496033},
  {'label': 'fear', 'score': 0.9281682372093201},
  {'label': 'sadness', 'score': 0.9671575427055359},
  {'label': 'joy', 'score': 0.9327967166900635},
  {'label': 'joy', 'score': 0.6528705954551697},
  {'label': 'neutral', 'score': 0.5494768023490906}]]

In [10]:
sentences = [s.strip() for s in books["description"].iloc[0].split(".") if s.strip()]

sentences

['A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives',
 'John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers',
 'It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up',
 'Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist',
 'He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption',
 'Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the wo

In [11]:
predictions = classifier(sentences)

In [12]:
predictions

[[{'label': 'surprise', 'score': 0.7296026349067688},
  {'label': 'neutral', 'score': 0.46625006198883057},
  {'label': 'neutral', 'score': 0.6978468298912048},
  {'label': 'fear', 'score': 0.9839729070663452},
  {'label': 'sadness', 'score': 0.9560651183128357},
  {'label': 'joy', 'score': 0.9490272998809814},
  {'label': 'joy', 'score': 0.6701956987380981}]]

In [13]:
sorted(predictions[0], key=lambda x: x["label"])

[{'label': 'fear', 'score': 0.9839729070663452},
 {'label': 'joy', 'score': 0.9490272998809814},
 {'label': 'joy', 'score': 0.6701956987380981},
 {'label': 'neutral', 'score': 0.46625006198883057},
 {'label': 'neutral', 'score': 0.6978468298912048},
 {'label': 'sadness', 'score': 0.9560651183128357},
 {'label': 'surprise', 'score': 0.7296026349067688}]

In [15]:
import numpy as np

emotion_labels = ["anger", "disgust", "fear", "joy", "neutral", "sadness", "surprise"]
isbn = []
emotion_scores = {label: [] for label in emotion_labels}
        

In [16]:
import re

def split_sentences(text: str):
    """Split on ., !, ? and drop empties/NaN."""
    if pd.isna(text):
        return []
    parts = re.split(r"[.!?]+", str(text))
    return [s.strip() for s in parts if s.strip()]

In [17]:
def max_scores_for_book(preds, labels):
    """Take the max score per label across all sentences."""
    # normalize: single-string output -> wrap to list[list[...]]
    if isinstance(preds, list) and preds and isinstance(preds[0], dict):
        preds = [preds]
    out = {lab: 0.0 for lab in labels}
    for sent_preds in preds or []:
        for d in sent_preds:
            lab = d.get("label", "").lower()
            if lab in out:
                s = float(d.get("score", 0.0))
                if s > out[lab]:
                    out[lab] = s
    return out

In [18]:
N = len(books)

from tqdm import tqdm

for i in tqdm(range(len(books))):
    desc = books.at[i, "description"]
    sents = split_sentences(desc)

    if not sents:
        max_scores = {lab: 0.0 for lab in emotion_labels}
    else:
        # light retry for API 429/503/etc.
        for attempt in range(3):
            try:
                preds = classifier(sents, top_k=len(emotion_labels))
                break
            except Exception:
                if attempt == 2:
                    preds = []  # fall back to zeros
                else:
                    time.sleep(2 * (attempt + 1))
        max_scores = max_scores_for_book(preds, emotion_labels)

    isbn.append(books.at[i, "isbn13"])
    for lab in emotion_labels:
        emotion_scores[lab].append(max_scores[lab])


100%|██████████| 5197/5197 [20:09<00:00,  4.30it/s]


In [19]:
emotion_scores

{'anger': [0.029641350731253624,
  0.6033434867858887,
  0.041300926357507706,
  0.3253040611743927,
  0.05883363261818886,
  0.20615458488464355,
  0.5513831973075867,
  0.023656265810132027,
  0.30066949129104614,
  0.01765836775302887,
  0.05883363261818886,
  0.5781001448631287,
  0.07590586692094803,
  0.04196577146649361,
  0.1264883428812027,
  0.3594604432582855,
  0.16215923428535461,
  0.004236503969877958,
  0.061681926250457764,
  0.20966701209545135,
  0.10191847383975983,
  0.02460910566151142,
  0.030959250405430794,
  0.022970035672187805,
  0.34315136075019836,
  0.06548014283180237,
  0.016607118770480156,
  0.057987030595541,
  0.05285235494375229,
  0.34632763266563416,
  0.06529022008180618,
  0.01876893825829029,
  0.019309133291244507,
  0.16889658570289612,
  0.004902247339487076,
  0.050679996609687805,
  0.04054473340511322,
  0.10138513147830963,
  0.11856506019830704,
  0.050533272325992584,
  0.5636680126190186,
  0.9127213358879089,
  0.0683567076921463,
 

In [20]:
emotion_df = pd.DataFrame(emotion_scores)
emotion_df["isbn13"] = isbn

In [21]:
emotion_df

,anger,disgust,fear,joy,neutral,sadness,surprise,isbn13
0,0.029641,0.338239,0.983973,0.949027,0.697847,0.956065,0.729603,9780002005883
1,0.603343,0.461991,0.930384,0.704422,0.891110,0.290987,0.212221,9780002261982
2,0.041301,0.024568,0.973285,0.767237,0.042176,0.010860,0.009796,9780006178736
3,0.325304,0.125763,0.436339,0.242209,0.732685,0.043272,0.029084,9780006280897
4,0.058834,0.152074,0.095043,0.199560,0.890048,0.475880,0.074878,9780006280934
...,...,...,...,...,...,...,...,...
5192,0.204142,0.053939,0.923713,0.303276,0.807058,0.974265,0.028639,9788172235222
5193,0.058449,0.127941,0.025688,0.400262,0.891073,0.016014,0.227765,9788173031014
5194,0.011246,0.010868,0.314810,0.942169,0.344739,0.060436,0.056820,9788179921623
5195,0.034505,0.080506,0.409870,0.776052,0.950763,0.317455,0.049227,9788185300535


In [22]:
books = pd.merge(books, emotion_df, on="isbn13")

In [23]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,...,title_and_subtitle,tagged_description,simplified_categories,anger,disgust,fear,joy,neutral,sadness,surprise
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,...,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction,0.029641,0.338239,0.983973,0.949027,0.697847,0.956065,0.729603
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,...,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,Fiction,0.603343,0.461991,0.930384,0.704422,0.891110,0.290987,0.212221
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,...,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction,0.041301,0.024568,0.973285,0.767237,0.042176,0.010860,0.009796
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,...,The Four Loves,9780006280897 Lewis' work on the nature of lov...,Nonfiction,0.325304,0.125763,0.436339,0.242209,0.732685,0.043272,0.029084
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,...,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",Nonfiction,0.058834,0.152074,0.095043,0.199560,0.890048,0.475880,0.074878
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,...,Mistaken Identity,9788172235222 On A Train Journey Home To North...,Fiction,0.204142,0.053939,0.923713,0.303276,0.807058,0.974265,0.028639
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,...,Journey to the East,9788173031014 This book tells the tale of a ma...,Nonfiction,0.058449,0.127941,0.025688,0.400262,0.891073,0.016014,0.227765
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,...,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...,Fiction,0.011246,0.010868,0.314810,0.942169,0.344739,0.060436,0.056820
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,...,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...,Nonfiction,0.034505,0.080506,0.409870,0.776052,0.950763,0.317455,0.049227


In [24]:
books.to_csv("books_with_emotions.csv", index=False)